# Input Perterbation

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




import pandas as pd

with open('u.data', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()

# Process the lines to create a list of tuples
data = []
for line in lines:
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    item_id = int(parts[1])
    rating = int(parts[2])
    # Skip the timestamp (parts[3])
    data.append((user_id, item_id, rating))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(data, columns=['userId', 'movieId', 'rating'])

# Print the first few rows of the DataFrame

df['movieId'].max()
df['movieId'].min()

temp = 1
userItem  = np.zeros((df['userId'].max()+1,df['movieId'].max()+1))
print(userItem )
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(944, 1683)


In [2]:
import pandas as pd
colName = ["movieId", "title", "Release Date", "Video Release Date",
           "IMDb URL", "Unknown", "Action", "Adventure", "Animation",
           "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
           "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
           "Thriller", "War", "Western"]

movies_df = pd.read_csv("u.item", sep="|", encoding="iso-8859-1",  names=colName)
movies_df.drop(columns= ["Release Date", "Video Release Date",
           "IMDb URL", "Unknown", "Action", "Adventure", "Animation",
           "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
           "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
           "Thriller", "War", "Western"], inplace=True)

In [3]:
movies_df

,movieId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [4]:
for i in range (len(df['userId'])):
  userItem[df.userId[i]][df.movieId[i]] = df.rating[i]

print(userItem)
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
(944, 1683)


In [5]:
R = userItem[1:]
print(R)
R = R[:, 1:]
print(R)

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

[[0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [6]:
A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)


In [20]:
def input_perterbation_sgd(R,A, d, gamma, lambd, epsilon):
    k = 50
    num_users, num_items = R.shape
    P = np.random.rand(num_users, d)
    Q = np.random.rand(num_items, d)
    R1 = R + np.random.laplace(scale=(4) / epsilon, size=R.shape)
    R0 = np.zeros((R.shape))
    B =1
    for i in range(R.shape[0]):
        for j in range (R.shape[1]): 
            if A[i,j] != 0: R0[i,j ] = R1[i,j]

    
    R0 = np.clip(R0, -B, B)
    # Assuming R is a numpy array, find non-zero (i.e., rated) entries
    rated_indices = np.argwhere(R > 0)

    for epoch in range(k):
        np.random.shuffle(rated_indices)  # Shuffle the observed ratings
        for i, j in rated_indices:
            e_ui = R[i, j] - np.dot(P[i], Q[j]) 
       

            # Update rule remains the same, but only applied to observed ratings
            P[i] += gamma * (e_ui * Q[j] - lambd * P[i])
            Q[j] += gamma * (e_ui * P[i] - lambd * Q[j])

    return P, Q , P.dot(Q.T)


In [21]:
p,q, r =  input_perterbation_sgd(R,A, 3,0.01,0.6,  epsilon = 10  )

In [22]:
print(r)

r

[[3.13402868 2.76924908 2.57461007 ... 1.64954871 2.40059227 2.56136848]
 [3.22052313 2.84567637 2.64566473 ... 1.69540705 2.46737042 2.63191471]
 [2.58887389 2.28754664 2.12676431 ... 1.36269052 1.98310857 2.11580194]
 ...
 [3.41322291 3.01594731 2.80396758 ... 1.7969993  2.61527151 2.78932125]
 [3.44493658 3.04396957 2.83002092 ... 1.81344454 2.63916941 2.815348  ]
 [2.93392673 2.59243779 2.4102252  ... 1.54449072 2.24811127 2.39760051]]


array([[3.13402868, 2.76924908, 2.57461007, ..., 1.64954871, 2.40059227,
        2.56136848],
       [3.22052313, 2.84567637, 2.64566473, ..., 1.69540705, 2.46737042,
        2.63191471],
       [2.58887389, 2.28754664, 2.12676431, ..., 1.36269052, 1.98310857,
        2.11580194],
       ...,
       [3.41322291, 3.01594731, 2.80396758, ..., 1.7969993 , 2.61527151,
        2.78932125],
       [3.44493658, 3.04396957, 2.83002092, ..., 1.81344454, 2.63916941,
        2.815348  ],
       [2.93392673, 2.59243779, 2.4102252 , ..., 1.54449072, 2.24811127,
        2.39760051]])

In [25]:
import math
def RMSE(actual, predicted):
    err = 0
    
    count = np.count_nonzero(actual)
    for i in range(actual.shape[0]):
        for j in range(actual.shape[1]):
            if actual[i][j] > 0:
                
                err += (actual[i][j] - predicted[i][j])**2
    
    if count == 0: "error is not calculated therefore count = 0 "
    else : 
        mse = err/count
    rmse = math.sqrt(mse)
    
    return  rmse
    
l =RMSE(R,r) 
print(l)

# input_perterbation_sgd(R,A, d, gamma, lambd, epsilon):

1.1072429544908493


In [9]:
ISGD_Rmse = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for _ in (ep):

    p,q , r = input_perterbation_sgd(R,A, 3,0.01,0.06,  epsilon = _  )

    temp = RMSE(R,r)
    ISGD_Rmse.append(temp)

In [ ]:
print(ISGD_Rmse)

In [39]:
import csv



csv_file_path = 'output2.csv'


In [42]:
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(['ISGD_Rmse'] + ISGD_Rmse)

In [ ]:



Re

In [42]:
import numpy as np

def non_Rated_movie(user_item_matrix, j):

    zero_indices = np.where(user_item_matrix[:, j] == 0)[0]
    ratings_dict = {}

    return zero_indices.tolist()


In [43]:
user =  5 



In [44]:
N  = non_Rated_movie(R,5)

In [45]:
print(N)

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,

In [69]:
def store_ratings(user_item_matrix, zero_indices, j):
    ratings_dict = {}
    for i in zero_indices:
        rating = user_item_matrix[i, j]
        ratings_dict[i] = rating
        
    top_20_ratings_dict = dict(list(ratings_dict.items())[:20])
    return top_20_ratings_dict

# Example usage:
# Assuming user_item_matrix_new is a numpy.ndarray representing your new user-item matrix
# and zero_indices is the list generated from the previous function
# and j is the column index you want to store ratings for
# ratings_dict = store_ratings(user_item_matrix_new, zero_indices, j)


In [70]:
k = store_ratings(r,N,user)

In [71]:
print(k)

{1: 3.2506139680398043, 2: 2.6130587956714546, 3: 3.7494548995584918, 4: 2.6355285922382525, 5: 2.975328650939413, 6: 3.570303529590045, 7: 3.3781947235545635, 9: 3.4056750810223404, 10: 3.1007311862566427, 11: 3.5352399845032862, 12: 2.923084193010322, 13: 3.454623303645149, 14: 2.706708883743525, 15: 3.6277633215449043, 16: 2.589171799507062, 18: 2.9936129753823497, 19: 2.5650812591811323, 20: 2.578821176911684, 21: 3.0292155653218287, 22: 3.0800261668443434}


In [107]:
def store_ratings_sorted(user_item_matrix, zero_indices, j):
    """
    Store ratings corresponding to column j and row indices in a dictionary sorted in descending order of keys.

    Parameters:
    - user_item_matrix (numpy.ndarray): 2D array representing the user-item matrix.
    - zero_indices (list): List of row indices where the element corresponding to column j is 0.
    - j (int): Column index.

    Returns:
    - dict: Dictionary with row indices as keys and ratings from the user-item matrix as values, sorted in descending order of keys.
    """
    ratings_dict = {}
    for i in zero_indices:
        rating = user_item_matrix[i, j]
        ratings_dict[i] = rating

    sorted_ratings_dict = dict(sorted(ratings_dict.items(), key=lambda x: x[1], reverse=True))
    return sorted_ratings_dict


In [132]:
m  =  store_ratings_sorted(r,N,1)

In [133]:
print(m)

{687: 3.871337075079043, 506: 3.6132254580499588, 848: 3.608786386636343, 426: 3.571661643599024, 809: 3.5182889234077757, 627: 3.4932203581810897, 476: 3.4774022182804933, 906: 3.4688778270288134, 117: 3.4507714619348864, 329: 3.4290131664730463, 938: 3.41426290358117, 471: 3.388889610571518, 927: 3.3618800146045444, 365: 3.3450858924110722, 518: 3.334864302482554, 810: 3.328597249325039, 151: 3.3270269114977165, 356: 3.3264526191183625, 371: 3.322389579549622, 522: 3.3204212058430387, 172: 3.3186671923867097, 33: 3.313235589712595, 241: 3.3086796324057226, 136: 3.3042550337290932, 512: 3.3012922527653137, 533: 3.298212971908339, 323: 3.296300555401191, 635: 3.291991895090513, 861: 3.2869174784250923, 3: 3.282360766153447, 350: 3.27998038452336, 269: 3.2723693344517844, 260: 3.2716827697547677, 564: 3.2714483427551446, 251: 3.2595929841693603, 366: 3.248803262459655, 922: 3.2465730922217317, 769: 3.2437381188017254, 163: 3.239973889602796, 340: 3.2366944303464784, 387: 3.2317094047333

In [110]:
top_20_ratings_dict = {}
count = 0
for key, value in m.items():
    if count < 20:
        top_20_ratings_dict[key] = value
        count += 1
    else:break

In [111]:
def get_movie_names(movies_df, top_20_ratings_dict):
    movie_names_dict = {}
    for key in top_20_ratings_dict.keys():
        movie_id = movies_df.loc[movies_df['movieId'] == key, 'title'].values[0]
        movie_names_dict[key] = movie_id
    return movie_names_dict


In [112]:
mo = get_movie_names(movies_df, top_20_ratings_dict)

In [120]:
def predRating(user,R,Rpred,movies_df):
    n = non_Rated_movie(R,user)
    r = store_ratings_sorted(Rpred,n,user)
    t20 = dict(list(r.items())[:20])
    


    mu = get_movie_names(movies_df, t20)
    return mu
    
    



In [121]:
mu = predRating(115,R,r,movies_df)

In [119]:
mu


{687: "McHale's Navy (1997)",
 506: 'Rebel Without a Cause (1955)',
 848: 'Murder, My Sweet (1944)',
 426: 'Transformers: The Movie, The (1986)',
 809: 'Rising Sun (1993)',
 627: 'Robin Hood: Prince of Thieves (1991)',
 476: 'First Wives Club, The (1996)',
 906: 'Oscar & Lucinda (1997)',
 117: 'Rock, The (1996)',
 329: 'Desperate Measures (1998)',
 938: 'Smile Like Yours, A (1997)',
 471: 'Courage Under Fire (1996)',
 927: 'Flower of My Secret, The (Flor de mi secreto, La) (1995)',
 365: 'Powder (1995)',
 518: "Miller's Crossing (1990)",
 810: 'Shadow, The (1994)',
 151: 'Willy Wonka and the Chocolate Factory (1971)',
 356: 'Client, The (1994)',
 371: 'Bridges of Madison County, The (1995)',
 172: 'Empire Strikes Back, The (1980)'}

In [152]:
import numpy as np

def predRatings(user_item_matrix, Rcap, j, movies_df):
    def non_Rated_movie(user_item_matrix, j):
        zero_indices = np.where(user_item_matrix[:, j] == 0)[0]
        return zero_indices.tolist()

    def store_ratings_sorted(Rcap, zero_indices, j):
        ratings_dict = {}
        for i in zero_indices:
            rating = Rcap[i, j]
            ratings_dict[i] = rating
        sorted_ratings_dict = dict(sorted(ratings_dict.items(), key=lambda x: x[1], reverse=True))
        sorted_ratings_dict = dict(list(sorted_ratings_dict.items())[:20])
        return sorted_ratings_dict

    def get_movie_names(movies_df, top_20_ratings_dict):
        movie_names_dict = {}
        for key in top_20_ratings_dict.keys():
            movie_id = movies_df.loc[movies_df['movieId'] == key, 'title'].values[0]
            movie_names_dict[key] = movie_id
        return movie_names_dict

    zero_indices = non_Rated_movie(user_item_matrix, j)
    top_20_ratings_dict = store_ratings_sorted(Rcap, zero_indices, j)
    movie_names_dict = get_movie_names(movies_df, top_20_ratings_dict)

    return movie_names_dict

In [153]:
p = predRatings(R,r,58,movies_df)

In [154]:
p

{687: "McHale's Navy (1997)",
 506: 'Rebel Without a Cause (1955)',
 848: 'Murder, My Sweet (1944)',
 426: 'Transformers: The Movie, The (1986)',
 809: 'Rising Sun (1993)',
 627: 'Robin Hood: Prince of Thieves (1991)',
 476: 'First Wives Club, The (1996)',
 906: 'Oscar & Lucinda (1997)',
 117: 'Rock, The (1996)',
 329: 'Desperate Measures (1998)',
 938: 'Smile Like Yours, A (1997)',
 471: 'Courage Under Fire (1996)',
 927: 'Flower of My Secret, The (Flor de mi secreto, La) (1995)',
 365: 'Powder (1995)',
 518: "Miller's Crossing (1990)",
 810: 'Shadow, The (1994)',
 151: 'Willy Wonka and the Chocolate Factory (1971)',
 356: 'Client, The (1994)',
 371: 'Bridges of Madison County, The (1995)',
 522: 'Down by Law (1986)'}

In [42]:
import streamlit as st
import pandas as pd

# Sample user-item matrix (replace this with your actual data)
user_item_matrix = pd.DataFrame({
    'user_id': [1, 2, 3, 4],
    'movie_id': [101, 102, 103, 104],
    'rating': [5, 4, 3, 5]
})

# Sample movies DataFrame (replace this with your actual data)
movies_df = pd.DataFrame({
    'movie_id': [101, 102, 103, 104],
    'title': ['Movie A', 'Movie B', 'Movie C', 'Movie D']
})

# Function to get recommendations for a given user ID
def get_recommendations(user_id, user_item_matrix, movies_df):
    # Replace this with your recommendation logic
    recommended_movies = user_item_matrix[user_item_matrix['user_id'] == user_id]['movie_id']
    recommended_movies = movies_df[movies_df['movie_id'].isin(recommended_movies)]
    return recommended_movies

# Streamlit UI
st.title('Movie Recommendation System')

user_id = st.number_input('Enter User ID', min_value=1, max_value=user_item_matrix['user_id'].max())

if st.button('Get Recommendations'):
    recommendations = get_recommendations(user_id, user_item_matrix, movies_df)
    if not recommendations.empty:
        st.subheader('Recommended Movies:')
        st.table(recommendations[['movie_id', 'title']])
    else:
        st.write('No recommendations found for the given user ID.')



2024-05-07 15:45:03.676 
  command:

    streamlit run /opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-05-07 15:45:03.678 Session state does not function when running a script without `streamlit run`


In [155]:



import numpy as np

# Assuming you have a matrix called 'my_matrix'
my_matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Save the matrix to a CSV file
np.savetxt('my_matrix.csv', my_matrix, delimiter=',')

In [156]:
import numpy as np

# Load the matrix from the CSV file
loaded_matrix = np.loadtxt('my_matrix.csv', delimiter=',')
print(loaded_matrix)

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
